In [1]:
import geopandas as gpd
from pyproj import Geod
from geopandas import clip
from shapely.geometry import LineString, MultiLineString

In [10]:
import geopandas as gpd
from pyproj import Geod
import matplotlib.pyplot as plt

# Load GeoJSON files

network = gpd.read_file("/content/VCTI_abc_network 5 1.geojson")
soil = gpd.read_file("/content/soil_data_be 3.geojson")





In [3]:
pip install fiona

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 107.1 MB/s eta 0:00:00


In [11]:
# Reproject to lat/lon to ensure they are in same crs
soil = soil.to_crs("EPSG:4326")
network = network.to_crs("EPSG:4326")


In [12]:

from geopandas import clip

network_clipped = gpd.clip(network, soil)


In [14]:
#Ellipsoidal length is the most accurate way to compute distances over the Earth’s surface using its ellipsoidal shape
from pyproj import Geod

# WGS84 ellipsoid (standard)
geod = Geod(ellps="WGS84")

def compute_ellipsoidal_length(geom):
    if geom.is_empty:
        return 0
    if geom.geom_type == 'MultiLineString':
        return sum(geod.geometry_length(part) for part in geom.geoms)
    elif geom.geom_type == 'LineString':
        return geod.geometry_length(geom)
    else:
        return 0


In [15]:
ellipsoidal_lengths = []

for geom in network_clipped.geometry:
    length = compute_ellipsoidal_length(geom)
    ellipsoidal_lengths.append(length)

network_clipped["ellipsoidal_length_m"] = ellipsoidal_lengths

In [25]:
print(network_clipped)

    id                                               icon           name  \
0   16  http://maps.google.com/mapfiles/kml/pushpin/yl...    SR-S19-Bore   
1   15  http://maps.google.com/mapfiles/kml/pushpin/yl...      SR-S18-RR   
2   14  http://maps.google.com/mapfiles/kml/pushpin/yl...   SR-S17-Water   
3   13  http://maps.google.com/mapfiles/kml/pushpin/yl...    SR-S16-Bore   
4   12  http://maps.google.com/mapfiles/kml/pushpin/yl...      SR-S15-HW   
5   11  http://maps.google.com/mapfiles/kml/pushpin/yl...    SR-S14-Bore   
6   10  http://maps.google.com/mapfiles/kml/pushpin/yl...  SR-S13-Trench   
7    9  http://maps.google.com/mapfiles/kml/pushpin/yl...   SR-S12-Water   
8    8  http://maps.google.com/mapfiles/kml/pushpin/yl...    SR-S11-Bore   
9    7  http://maps.google.com/mapfiles/kml/pushpin/yl...   SR-S10-Water   
10   6  http://maps.google.com/mapfiles/kml/pushpin/yl...    SR-S09-Bore   
11   5  http://maps.google.com/mapfiles/kml/pushpin/yl...      SR-S08-RR   
12   4  http

In [20]:
network_clipped["ellipsoidal_length_m"] = network_clipped.geometry.apply(compute_ellipsoidal_length)

# View results
print(network_clipped[["ellipsoidal_length_m"]].head())


   ellipsoidal_length_m
0           5707.926406
1            123.688737
2            131.451586
3          11910.979025
4             81.769053
